In [2]:
import pandas as pd
from IPython.core.display import HTML
from IPython.core.display import Image 
%load_ext music21.ipython21.ipExtension
import numpy as np
import music21 as mu
import pymongo
%pylab inline
from pymongo import MongoClient
from bs4 import BeautifulSoup
import re
import pydash as py_
pd.set_option('display.mpl_style', 'default')
plt.rcParams['figure.figsize'] = (15, 5)
import math
import music21 as mu
from functools import partial
import itertools
# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)


Populating the interactive namespace from numpy and matplotlib


In [3]:
muAutumnLeaves = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KJAutumnLeaves1996_1371_192.xml')
muStellaByStarlight = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KJStellaByStarlight1983_1534_161.xml')
muGroovinHigh = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KJGroovinHigh1999_1996_290.xml')
muAllTheThingsYouAre = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KJAllTheThingsYouAre1983_2143_290.xml')
muIfIWereABell = mu.converter.parse('FinalDataSet/XMLFiles/JazzSolos/KJIfIWereABell2002_2161_227.xml')

In [4]:
df = pd.read_pickle('data_set_as_pk.pkl')

<h4><a id="microphrase">Similar phrases, sequences and patterns</a></h4>

<div>
similiar phrases seem to be in these corposes, so if we don't look for idetnical repetition, but similiartity what is found. This tells us about repetition in a different way, patterns also are revealed - looking at longer phrases
</div>

<div>2 note phrases first what is happening....</div>

In [5]:
#utilitity methods to build phrases
def titleAsString(data_row):
    titleAsString = str(data_row[0][0:1][0])
    locationAsString = str(data_row[0][1:2][0])
    return titleAsString + ', measure: ' + locationAsString

def concatenatePhrase(x, data_row):
    phraseArray = []
    startingIndexForNoteDetails = x[1]
    endingIndexForNoteDetails = x[2]
    
    for i in range(0, x[0]):
        phraseArray.append(data_row[i][startingIndexForNoteDetails:endingIndexForNoteDetails])

    return phraseArray

def flagIfNoFrequency(x, data_row):
    flag = 0
    for i in range(0, x):
        if np.isnan(data_row[i][2]):
            flag = 1
    return flag

In [6]:
#class to break notes up into individual phrases and track different lengths of phrases
class CreateDataFramesForTwoNotePhraseAnalysis:
    def __init__(self, dataFrame="", numberOfNotesInPhrase = -1):
        self.df = dataFrame
        self.microPhraseArray = -1
        self.twoNotePhrases = -1
        self.startValueToIncludeFrequencyInPhrase = 2
        self.startValueToExludeFrequency = 3
        self.endValueToExcludeRelativeFrequency = 4
        self.endValueToIncludeAllNoteDetails = 5
        self.numberOfNotesInPhrase = numberOfNotesInPhrase

    def createDataFrames(self):
        dfForPhrasePatterns = self.df[self.df['part'] == 'P1'][['title','measure', 'midiNumber', 'quarterNoteDuration', 'nextNoteFreqDistance', 'noteEventStartingLocation', 'Location in measure']]
        listOfNotes = dfForPhrasePatterns.values.tolist()
        microPhraseLength = self.numberOfNotesInPhrase
        microPhraseArray = []
        microPhrase = []
        for x in range(0, len(listOfNotes)):
            counter = 0
            while(counter < microPhraseLength):
                try:
                    val = listOfNotes[x + counter]
                except:
                    pass
                microPhrase.append(val)
                counter = counter + 1
            microPhraseArray.append(microPhrase)
            microPhrase = []
            
        self.twoNotePhrases = pd.DataFrame(microPhraseArray)

        self.twoNotePhrases['Title and location'] = self.twoNotePhrases.apply(titleAsString, axis=1)
        self.twoNotePhrases['Phrase with relative frequency only'] = self.twoNotePhrases.apply(partial(concatenatePhrase, (self.numberOfNotesInPhrase, self.endValueToExcludeRelativeFrequency, self.endValueToIncludeAllNoteDetails)), axis=1)
        self.twoNotePhrases['No Frequency'] = self.twoNotePhrases.apply(partial(flagIfNoFrequency, self.numberOfNotesInPhrase), axis=1)

    def removeGhostPhrasesAndUnneededColumns(self):
        self.twoNotePhrases = self.twoNotePhrases[self.twoNotePhrases['No Frequency'] == 0]
        for i in range(0, self.numberOfNotesInPhrase):
            del self.twoNotePhrases[i]
        del self.twoNotePhrases['No Frequency']

    def returnDataFramesForAnalysis(self):
        #print self.twoNotePhrases
        return self.twoNotePhrases


In [7]:

createFiveNotePhraseData = CreateDataFramesForTwoNotePhraseAnalysis(df, 5)
createFiveNotePhraseData.createDataFrames()
createFiveNotePhraseData.removeGhostPhrasesAndUnneededColumns()
fiveNotephrases = createFiveNotePhraseData.returnDataFramesForAnalysis()



In [8]:
fiveNotephrases['Phrase with relative frequency only'][0:1].values[0][0]

[2.0]

<div>So what does one of these look like?</div>

In [9]:
select = muAllTheThingsYouAre.parts[0].measures(32,33)
#select.show()

In [10]:
def createFlexiblePhrases(data_row):
    retArr = []
    for item in data_row['Phrase with relative frequency only']:
        arr = []
        arr.append(item[0] - 1)
        arr.append(item[0])
        arr.append(item[0] + 1)
        retArr.append(arr)
                
    queryArray = []
    for x in itertools.product(*retArr):
        queryArray.append(x)
                
        
    return queryArray

fiveNotephrases['Structrues of related phrases'] = fiveNotephrases.apply(createFlexiblePhrases, axis=1)

In [11]:
flexQueries = fiveNotephrases['Structrues of related phrases'].values

In [17]:
fiveNotephrases

,Title and location,Phrase with relative frequency only,Structrues of related phrases
13,"Stella By Starlight, measure: 3.0","[[2.0], [9.0], [-2.0], [-3.0], [-4.0]]","[(1.0, 8.0, -3.0, -4.0, -5.0), (1.0, 8.0, -3.0..."
14,"Stella By Starlight, measure: 3.0","[[9.0], [-2.0], [-3.0], [-4.0], [-2.0]]","[(8.0, -3.0, -4.0, -5.0, -3.0), (8.0, -3.0, -4..."
15,"Stella By Starlight, measure: 3.0","[[-2.0], [-3.0], [-4.0], [-2.0], [-1.0]]","[(-3.0, -4.0, -5.0, -3.0, -2.0), (-3.0, -4.0, ..."
16,"Stella By Starlight, measure: 4.0","[[-3.0], [-4.0], [-2.0], [-1.0], [nan]]","[(-4.0, -5.0, -3.0, -2.0, nan), (-4.0, -5.0, -..."
23,"Stella By Starlight, measure: 5.0","[[1.0], [1.0], [2.0], [1.0], [-1.0]]","[(0.0, 0.0, 1.0, 0.0, -2.0), (0.0, 0.0, 1.0, 0..."
24,"Stella By Starlight, measure: 5.0","[[1.0], [2.0], [1.0], [-1.0], [-2.0]]","[(0.0, 1.0, 0.0, -2.0, -3.0), (0.0, 1.0, 0.0, ..."
25,"Stella By Starlight, measure: 5.0","[[2.0], [1.0], [-1.0], [-2.0], [-2.0]]","[(1.0, 0.0, -2.0, -3.0, -3.0), (1.0, 0.0, -2.0..."
26,"Stella By Starlight, measure: 5.0","[[1.0], [-1.0], [-2.0], [-2.0], [-1.0]]","[(0.0, -2.0, -3.0, -3.0, -2.0), (0.0, -2.0, -3..."
27,"Stella By Starlight, measure: 5.0","[[-1.0], [-2.0], [-2.0], [-1.0], [-2.0]]","[(-2.0, -3.0, -3.0, -2.0, -3.0), (-2.0, -3.0, ..."
28,"Stella By Starlight, measure: 5.0","[[-2.0], [-2.0], [-1.0], [-2.0], [2.0]]","[(-3.0, -3.0, -2.0, -3.0, 1.0), (-3.0, -3.0, -..."


In [16]:
for stuff in flexQueries[0:1]:
    print stuff

[(1.0, 8.0, -3.0, -4.0, -5.0), (1.0, 8.0, -3.0, -4.0, -4.0), (1.0, 8.0, -3.0, -4.0, -3.0), (1.0, 8.0, -3.0, -3.0, -5.0), (1.0, 8.0, -3.0, -3.0, -4.0), (1.0, 8.0, -3.0, -3.0, -3.0), (1.0, 8.0, -3.0, -2.0, -5.0), (1.0, 8.0, -3.0, -2.0, -4.0), (1.0, 8.0, -3.0, -2.0, -3.0), (1.0, 8.0, -2.0, -4.0, -5.0), (1.0, 8.0, -2.0, -4.0, -4.0), (1.0, 8.0, -2.0, -4.0, -3.0), (1.0, 8.0, -2.0, -3.0, -5.0), (1.0, 8.0, -2.0, -3.0, -4.0), (1.0, 8.0, -2.0, -3.0, -3.0), (1.0, 8.0, -2.0, -2.0, -5.0), (1.0, 8.0, -2.0, -2.0, -4.0), (1.0, 8.0, -2.0, -2.0, -3.0), (1.0, 8.0, -1.0, -4.0, -5.0), (1.0, 8.0, -1.0, -4.0, -4.0), (1.0, 8.0, -1.0, -4.0, -3.0), (1.0, 8.0, -1.0, -3.0, -5.0), (1.0, 8.0, -1.0, -3.0, -4.0), (1.0, 8.0, -1.0, -3.0, -3.0), (1.0, 8.0, -1.0, -2.0, -5.0), (1.0, 8.0, -1.0, -2.0, -4.0), (1.0, 8.0, -1.0, -2.0, -3.0), (1.0, 9.0, -3.0, -4.0, -5.0), (1.0, 9.0, -3.0, -4.0, -4.0), (1.0, 9.0, -3.0, -4.0, -3.0), (1.0, 9.0, -3.0, -3.0, -5.0), (1.0, 9.0, -3.0, -3.0, -4.0), (1.0, 9.0, -3.0, -3.0, -3.0), (1.0, 9.0

In [23]:
fiveNotephrases['Structrues of related phrases'][0:2]

13    [(1.0, 8.0, -3.0, -4.0, -5.0), (1.0, 8.0, -3.0...
14    [(8.0, -3.0, -4.0, -5.0, -3.0), (8.0, -3.0, -4...
Name: Structrues of related phrases, dtype: object